In [3]:
import numpy as np
import math
import statistics
import tensorflow as tf
import string
import random
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from google.colab import drive
from tensorflow.python.framework import ops
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import Model
from tensorflow.keras.layers.experimental import preprocessing


# We mount our Google Drive that contains the datasets
drive.mount('/content/gdrive')




#### The function for reading and parsing file
def read_seq(sequence):
  f = open(sequence, 'r')
  train = []
  trainlabel = []

    # Reading file and extracting paths and labels
  with open(sequence, 'r') as File:
    infoFile = File.readlines() # Reading all the lines from File
    count = 0
      
    for line in infoFile: #Reading line-by-line and spliting them
      split_line = line.split() 
      
      # training data addin
      charList = list(split_line[1])
      seq = []
          
      # if for some reason the protein sequence is less than 1000 char long
      if len(charList) < 1000:
        for i in range(len(charList)):
          seq.append(ord(charList[i]) - ord('A') + 1)
            
        while len(seq) < 1000:
          seq.append(0)
            
        # grab the labels and convert seq into a one hot encoding of 21
        if seq[len(seq)-1] > 20:
            seq[len(seq)-1] = 20

        seq = tf.one_hot(seq,21)
        train.append(seq)

        if split_line[0] == 'delta':
          trainlabel.append(0.0)
        else if split_line[0] == 'reg':
          trainlabel.append(1.0)
        else if split_line[0] == 'beta':
          trainlabel.append(2.0)
        else if split_line[0] == 'alpha':
          trainlabel.append(3.0)

        seq = []

      else:
        for i in range(1000): # convert each letter into an int
          seq.append(ord(charList[i]) - ord('A') + 1)
            
          # grab the labels and convert seq into a one hot encoding of 21
        if seq[len(seq)-1] > 20:
            seq[len(seq)-1] = 20

        seq = tf.one_hot(seq,21)
        train.append(seq)

        if split_line[0] == 'delta':
          trainlabel.append(0.0)
        else if split_line[0] == 'reg':
          trainlabel.append(1.0)
        else if split_line[0] == 'beta':
          trainlabel.append(2.0)
        else if split_line[0] == 'alpha':
          trainlabel.append(3.0)

        seq = []

      count += 1

  return train, trainlabel


############################## use pretrained model to generate sequences that can be read by THIS model #################################################

x_train, y_train = read_seq('/content/gdrive/My Drive/proteinStuff/train.txt')
x_test, y_test = read_seq('/content/gdrive/My Drive/proteinStuff/test.txt')


# convert data read 
y_train = ops.convert_to_tensor(y_train, dtype=tf.string)
y_test = ops.convert_to_tensor(y_test, dtype=tf.string)


x_train = ops.convert_to_tensor(x_train, dtype=tf.float32)
x_test = ops.convert_to_tensor(x_test, dtype=tf.float32)

# load model and test it against the COVID variants
model = keras.models.load_model('/content/gdrive/My Drive/saved_model08102021') # this model is just filler for now
historyMod = model.fit(x=x_train, y=y_train, epochs=20, batch_size=28, validation_data=(x_test, y_test))


# We print out a summary of our model
model.summary()

# We plot results of the training model
plt.plot(historyMod.history['accuracy'], label='Training data')
plt.plot(historyMod.history['val_accuracy'], label='Validation data')
plt.title('Accuracy training vs. Accuracy validation')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(loc="upper left")
plt.show()

# We plot the loss
plt.plot(historyMod.history['loss'], label='Training data')
plt.plot(historyMod.history['val_loss'], label='Validation data')
plt.title('Loss training vs. Loss validation')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(loc="upper left")
plt.show()

# This portion takes care of the perplexity plotting
perplexity = tf.exp(historyMod.history['loss'])
plt.plot(perplexity)
plt.title('Perplexity of RNN')
plt.show()




######################### the next step would be for the model to generate the next 900 amino acids and see if it can be recognized as a COVID variant



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Epoch 1/20


UnimplementedError: ignored